In [41]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [136]:
def rank(text):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    scores=dict()
    counts=dict()
    contextSizes=dict()
    for i in range(len(filtered_words)):
        counts.setdefault(filtered_words[i],0)
        counts[filtered_words[i]]+=1
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        contextSizes.setdefault(filtered_words[i],contextSize)
        contextSizes[filtered_words[i]]=max(contextSizes[filtered_words[i]],contextSize)
        for j in range(i+1,rightBound+1):
            scores.setdefault(filtered_words[i],0)
            scores.setdefault(filtered_words[j],0)
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            scores[filtered_words[i]]+=similarity_score
            scores[filtered_words[j]]+=similarity_score
    wordScores=list(map(list, scores.items()))
    for i in range(len(wordScores)):
        wordScores[i][1]=counts[wordScores[i][0]]*wordScores[i][1]/(contextSizes[wordScores[i][0]])
    wordScores=sorted(wordScores, key = lambda x: x[1],reverse=True)
    print(contextSizes)
    return wordScores

In [137]:
rank("Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to perform a specific task without using explicit instructions, relying on patterns and inference instead. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop a conventional algorithm for effectively performing the task.")

{'word': 7, 'embedding': 7, 'collective': 6, 'name': 7, 'set': 7, 'language': 7, 'modeling': 7, 'feature': 7, 'learning': 7, 'technique': 7, 'natural': 7, 'processing': 7, '(': 7, 'nlp': 7, ')': 7, 'phrase': 7, 'vocabulary': 7, 'mapped': 7, 'vector': 7, 'real': 7, 'number': 7, '.': 7, 'conceptually': 7, 'involves': 7, 'mathematical': 7, 'space': 7, 'many': 7, 'dimension': 7, 'per': 7, 'continuous': 7, 'much': 7, 'lower': 6}


[['word', 1.430739133485726],
 ['language', 0.7497066190200192],
 ['space', 0.6977222583123616],
 ['embedding', 0.5931331451450076],
 ['dimension', 0.5623353991125312],
 ['vector', 0.36693941089989884],
 ['learning', 0.25229665637016296],
 ['technique', 0.22257279923983983],
 ['phrase', 0.22173083147832326],
 ['conceptually', 0.19739903190306254],
 ['vocabulary', 0.19167553527014597],
 ['name', 0.18996242646660125],
 ['.', 0.17516866466030478],
 ['modeling', 0.17400755733251572],
 ['involves', 0.17095569601016386],
 [')', 0.16447184021983827],
 ['feature', 0.16130013231720244],
 ['much', 0.16102361546031066],
 ['(', 0.15668801936720098],
 ['mathematical', 0.15417887403496675],
 ['set', 0.14678641515118734],
 ['collective', 0.14620503410696983],
 ['nlp', 0.14367199476276124],
 ['natural', 0.14298856684139796],
 ['processing', 0.1341869639498847],
 ['continuous', 0.13282889766352518],
 ['number', 0.1286993505699294],
 ['real', 0.12402210278170449],
 ['lower', 0.11843609996140003],
 ['map